# Imports

< In the table of contents, right-click on "Imports" and click "Select and Run Cells for this heading" to get started

In [1]:
%load_ext autoreload
%autoreload 2

from jupyter_cadquery import set_defaults, show, Camera, Collapse
from build123d import *
import math
from enum import Enum

set_defaults(
	viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


# Variables

In [2]:
SCREW_SHAFT_DIAM = 5
SCREW_HEAD_DIAM = 8.6
SCREW_HEAD_LENGTH = 5.2
SCREW_SHAFT_LENGTH = 25
WASHER_THICKNESS = 1
NUT_DIAM = 7.9

In [3]:
SCREW_HEAD_RADIUS = SCREW_HEAD_DIAM / 2
SCREW_SHAFT_RADIUS = SCREW_SHAFT_DIAM / 2
NUT_RADIUS = NUT_DIAM / 2

GOLD=(232, 176, 36)
GRAY=(170, 170, 170)

In [5]:
from components.fasteners.hex import HexBoreHole

HexBoreHole(SCREW_SHAFT_RADIUS, SCREW_HEAD_RADIUS, SCREW_HEAD_LENGTH, SCREW_SHAFT_LENGTH)

+


# Class Definitions

In [6]:
class BoltHeadType(Enum):
    Socket = 0
    Flat = 1
    Round = 2
    Hex = 3

class NutType(Enum):
    Hex = 0
    Cap = 1
    Wing = 2
    Square = 3
    TSlot = 4

class WasherType(Enum):
    Flat = 0
    SplitLock = 1
    Square = 2

class DriveType(Enum):
    Hex = 0
    Square = 1
    Star = 2
    Flat = 3

In [7]:
class Nut(BasePartObject):
    def __init__(
        self,
        outer_radius,
        inner_radius, 
        thickness,
        nut_type: NutType = NutType.Hex,
        rotation: VectorLike = (0,0,0),
        mode : Mode = Mode.ADD
    ):
        with BuildPart() as b:
            with BuildSketch() as hex_profile:
                match nut_type:
                    case NutType.Hex | _:
                        RegularPolygon(outer_radius, 6, major_radius=False)
                Circle(inner_radius, mode=Mode.SUBTRACT)
            extrude(amount=thickness)
        super().__init__(part=b.part, rotation=rotation, mode=mode)

In [8]:
class Washer(BasePartObject):
    def __init__(
        self,
        outer_radius,
        inner_radius,
        thickness,
        washer_type: WasherType = WasherType.Flat,
        rotation: VectorLike = (0,0,0),
        mode: Mode = Mode.ADD
    ):
        with BuildPart() as b:
            Cylinder(outer_radius, thickness, align=(Align.CENTER, Align.CENTER, Align.MIN))
            Cylinder(inner_radius, thickness, mode=Mode.SUBTRACT, align=(Align.CENTER, Align.CENTER, Align.MIN))
        super().__init__(part=b.part, rotation=rotation, mode=mode)

In [9]:
class Bolt(BasePartObject):
    def __init__(
        self,
        shaft_length,
        shaft_radius,
        head_length,
        head_radius,
        head_type: BoltHeadType = BoltHeadType.Socket,
        integrated_washer: bool = False,
        thread: bool = False,
        rotation: VectorLike = (0,0,0),
        mode : Mode = Mode.ADD
    ):
        with BuildPart() as b:
            match head_type:
                case BoltHeadType.Flat:
                    Cone(shaft_radius, head_radius, head_length, align=(Align.CENTER, Align.CENTER, Align.MIN))
                case BoltHeadType.Round:
                    with BuildSketch(Plane.XZ):
                        with BuildLine():
                            EllipticalCenterArc((0,0), head_radius, head_length)
                            Line((0,0), (head_radius, 0))
                            Line((0,0), (0, head_length))
                        make_face()
                    revolve(axis=Axis.Z)
                case BoltHeadType.Hex:
                    with BuildSketch():
                        RegularPolygon(head_radius, 6, major_radius=False)
                    extrude(amount=head_length)
                case BoltHeadType.Socket | _:
                    Cylinder(
                        head_radius,
                        head_length,
                        align=(Align.CENTER, Align.CENTER, Align.MIN)
                    )
            Cylinder(
                shaft_radius,
                shaft_length,
                align=(Align.CENTER, Align.CENTER, Align.MAX)
            )
        super().__init__(part=b.part, rotation=rotation, mode=mode)

In [10]:
class HexBoreHole(BasePartObject):
    def __init__(
        self,
        radius: float,
        hex_radius: float,
        counter_bore_depth: float,
        depth: float | None,
        rotation: VectorLike = (0, 0, 0),
        mode: Mode = Mode.SUBTRACT
    ):
        context = BuildPart._get_context(self)

        self.radius = radius
        self.hex_radius = hex_radius
        self.counter_bore_depth = counter_bore_depth
        if depth is not None:
            self.hole_depth = depth
        elif depth is None and context is not None:
            self.hole_depth = context.max_dimension
        else:
            raise ValueError("No depth provided")

        with BuildPart() as b:
            with BuildPart() as hex_nut:
                with BuildSketch():
                    RegularPolygon(self.hex_radius, 6)
                extrude(amount=-self.counter_bore_depth)
            Cylinder(self.radius, self.hole_depth, align=(Align.CENTER, Align.CENTER, Align.MAX))
        super().__init__(part=b.part, rotation=rotation, mode=mode)

# Scratch

In [11]:
show(
    Washer(SCREW_HEAD_DIAM, SCREW_SHAFT_RADIUS, WASHER_THICKNESS),
    Plane.XY.offset(WASHER_THICKNESS) * Bolt(SCREW_SHAFT_LENGTH, SCREW_SHAFT_RADIUS, SCREW_HEAD_LENGTH / 2, SCREW_HEAD_RADIUS, head_type=BoltHeadType.Round),
    Plane.XY.offset(-SCREW_SHAFT_LENGTH + WASHER_THICKNESS) * Nut(NUT_RADIUS, SCREW_SHAFT_RADIUS, SCREW_HEAD_LENGTH / 2),
    names=["Washer", "Bolt", "Nut"],
    colors=[GRAY, GRAY, GRAY]
)

+++
